## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import re

## Importing CSV file

In [2]:
dataDF = pd.read_csv("customer_data.csv",index_col="customer_id")
dataDF

,first_name,last_name,mobile_number,email,gender,car_make,car_model
customer_id,,,,,,,
1,Catriona,Alven,283-577-5685,calven0@mozilla.com,Male,Pontiac,Firebird
2,Alistair,Koppke,617-988-5980,akoppke1@unc.edu,Male,Pontiac,Firefly
3,Krishna,Merrywether,588-930-9381,kmerrywether2@blog.com,Male,GMC,Jimmy
4,Lona,Wilfling,623-945-0973,lwilfling3@msn.com,Female,Pontiac,Turbo Firefly
5,Jody,Hordell,186-673-9010,jhordell4@ifeng.com,Female,Mercury,Mountaineer
...,...,...,...,...,...,...,...
996,Neala,Adamowitz,410-377-6016,nadamowitzrn@4shared.com,Female,Dodge,Grand Caravan
997,Flora,Josham,796-126-4819,fjoshamro@nydailynews.com,Male,Mazda,Mazda6
998,Efren,MacVaugh,106-886-9230,emacvaughrp@1688.com,Male,BMW,7 Series


In [3]:
# information about columns
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     1000 non-null   object
 1   last_name      1000 non-null   object
 2   mobile_number  982 non-null    object
 3   email          982 non-null    object
 4   gender         982 non-null    object
 5   car_make       1000 non-null   object
 6   car_model      1000 non-null   object
dtypes: object(7)
memory usage: 62.5+ KB


In [4]:
# checking sample
dataDF.sample(8)

,first_name,last_name,mobile_number,email,gender,car_make,car_model
customer_id,,,,,,,
362,Yvonne,Pareman,880-718-6968,yparemana1@upenn.edu,Male,Mitsubishi,Truck
547,Noelle,Altofts,980-624-9772,naltoftsf6@springer.com,Male,Ford,Econoline E350
467,Tomi,Hovey,129-718-1716,thoveycy@freewebs.com,Female,Volvo,960
631,Piotr,Gorham,802-910-9208,pgorhamhi@bing.com,Male,Chrysler,Aspen
940,Wallie,MacClure,547-502-0249,wmacclureq3@sourceforge.net,Female,Mitsubishi,Galant
843,Orv,Issac,654-115-4316,oissacne@list-manage.com,Female,Chevrolet,Beretta
983,Daffy,Leyborne,174-574-8235,dleybornera@ihg.com,Male,Saab,900
346,Greg,Chicchetto,793-506-7756,gchicchetto9l@goo.ne.jp,Female,Chevrolet,Express 1500


## Cleaning data

In [5]:
dataDF.shape    # (1000,7)
dataDF.value_counts()

first_name  last_name   mobile_number  email                          gender       car_make       car_model       
Aarika      Hebson      850-610-3079   ahebsonbq@apple.com            Female       Mercedes-Benz  300E                1
Matt        Birkenshaw  176-315-6526   mbirkenshawlo@bbb.org          Female       Pontiac        Montana             1
Maude       MacElroy    717-992-3518   mmacelroy2r@accuweather.com    Female       Eagle          Talon               1
Maura       Kerrich     888-715-9953   mkerrich9g@japanpost.jp        Male         Nissan         Pathfinder          1
Maureene    Bagshawe    643-490-7773   mbagshawe19@sourceforge.net    Polygender   Nissan         Titan               1
                                                                                                                     ..
Flemming    Ambrogini   312-370-5351   fambroginilb@odnoklassniki.ru  Female       Volvo          S60                 1
Flora       Josham      796-126-4819   fjosha

length shown is 982 and not 1000
therefore null rows present

### 1. Null values

In [6]:
# dataDF[dataDF.isnull()==True]
null_count = dataDF.isnull().sum()
null_count

first_name        0
last_name         0
mobile_number    18
email            18
gender           18
car_make          0
car_model         0
dtype: int64

3 columns have 18 NaN each

#### Removing null values

In [7]:
# cannot drop whole row for NaN mobile and email, might affect data analysis
# mobile number and email is not mandatory for statistics, therefore zero value will not affect analysis

dataDF.loc[:,'mobile_number'].isnull().sum() # before filling = 18
dataDF.mobile_number = dataDF.loc[:,'mobile_number'].fillna(0) # filling
dataDF.loc[:,'mobile_number'].isnull().sum() # after filling

0

In [8]:
# cannot drop whole row for NaN mobile and email, might affect data analysis
# mobile number and email is not mandatory for statistics, therefore zero value will not affect analysis

dataDF.loc[:,'email'].isnull().sum() # before filling = 18
dataDF.email = dataDF.loc[:,'email'].fillna("") # filling
dataDF.loc[:,'email'].isnull().sum() # after filling

0

In [9]:
# gender might be required for statistics
# no way to predict gender based on other columns or rows
# better to drop

dataDF.loc[:,'gender'].isnull().sum() # before dropping = 18
dataDF = dataDF[dataDF['gender'].notna()]
dataDF.loc[:,'gender'].isnull().sum() # after dropping

0

#### Email ID

### 2. Formatting

#### Email ID

In [10]:
regex_expression_email = re.compile(r'^[a-zA-Z0-9_.]+@[a-zA-Z0-9\-]+\.[a-zA-Z_.]+$')

bool_values = dataDF.email.apply(lambda x: bool(regex_expression_email.match(x)))

bool_values = pd.concat([bool_values, dataDF.email], axis=1)
bool_values.columns=['bool_value','email']
bool_values.loc[bool_values.bool_value==False]

,bool_value,email
customer_id,,


bool_values is empty => all emails are in appropriate format

#### Mobile Number

In [11]:
dataDF.mobile_number.isnull().unique()
# no null entries

abc = dataDF.mobile_number.str.split('-')
abc = abc.str.join('')
abc
# now no more dashes in mobile numbers

dataDF.loc[:,('modified_mobile_number')] = abc.values
dataDF

C:\Users\Binayak\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,first_name,last_name,mobile_number,email,gender,car_make,car_model,modified_mobile_number
customer_id,,,,,,,,
1,Catriona,Alven,283-577-5685,calven0@mozilla.com,Male,Pontiac,Firebird,2835775685
2,Alistair,Koppke,617-988-5980,akoppke1@unc.edu,Male,Pontiac,Firefly,6179885980
3,Krishna,Merrywether,588-930-9381,kmerrywether2@blog.com,Male,GMC,Jimmy,5889309381
4,Lona,Wilfling,623-945-0973,lwilfling3@msn.com,Female,Pontiac,Turbo Firefly,6239450973
5,Jody,Hordell,186-673-9010,jhordell4@ifeng.com,Female,Mercury,Mountaineer,1866739010
...,...,...,...,...,...,...,...,...
996,Neala,Adamowitz,410-377-6016,nadamowitzrn@4shared.com,Female,Dodge,Grand Caravan,4103776016
997,Flora,Josham,796-126-4819,fjoshamro@nydailynews.com,Male,Mazda,Mazda6,7961264819
998,Efren,MacVaugh,106-886-9230,emacvaughrp@1688.com,Male,BMW,7 Series,1068869230


In [14]:
regex_expression_mobno = re.compile(r'^[0-9]{10}')

bool_values2 = dataDF.modified_mobile_number.apply(lambda x: bool(regex_expression_mobno.match(x)))

bool_values2 = pd.concat([bool_values2, dataDF.mobile_number], axis=1)
bool_values2.columns=['bool_value','mobno']
bool_values2.loc[bool_values2.bool_value==False]

,bool_value,mobno
customer_id,,


bool_values is empty => all mobile numbers are in appropriate format